In [1]:
import numpy as np
from sklearn.base import clone
from sklearn.dummy import DummyRegressor
from joblib import Parallel, delayed

class LOCI:
    def __init__(self, estimator, random_state=None, loss=None, n_jobs=1):
        self.estimator = estimator
        self.random_state = random_state
        self.loss = loss
        self.n_jobs = n_jobs
        self.feature_names_ = None
        self.X_train_ = None
        self.y_train_ = None

    def fit(self, X, y):
        self.X_train_ = X
        self.y_train_ = y
        self.feature_names_ = X.columns
        return self

    def _score_single_feature(self, j, X_test, y_test, v0):
        fname = self.feature_names_[j]
        model_j = clone(self.estimator)
        model_j.fit(self.X_train_[[fname]], self.y_train_)
        preds_j = model_j.predict(X_test[[fname]])
        vj = self.loss(y_test, preds_j)
        return fname, v0 - vj

    def score(self, X_test, y_test):
        if self.X_train_ is None:
            raise ValueError("You must call `fit` before `score`.")

        # Baseline: constant (mean) model
        dummy = DummyRegressor(strategy="mean")
        dummy.fit(self.X_train_, self.y_train_)
        pred_dummy = dummy.predict(np.zeros((len(X_test), 1)))  # doesn't use features
        v0 = self.loss(y_test, pred_dummy)

        # LOCI: leave-one-covariate-in
        results = Parallel(n_jobs=self.n_jobs)(
            delayed(self._score_single_feature)(j, X_test, y_test, v0)
            for j in range(X_test.shape[1])
        )

        return dict(results)


In [2]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error
from sklearn.datasets import load_diabetes
from sklearn.model_selection import train_test_split
import pandas as pd

# Load dataset
data = load_diabetes()
X = pd.DataFrame(data.data, columns=data.feature_names)
y = data.target
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

# Create LOCI object
loci = LOCI(
    estimator=RandomForestRegressor(random_state=42),
    random_state=42,
    loss=mean_squared_error,
    n_jobs=-1,
)

loci.fit(X_train, y_train)
loci_importance = loci.score(X_test, y_test)

print("LOCI Importances:\n", loci_importance)


LOCI Importances:
 {'age': -270.82926799915003, 'sex': 27.756080827185542, 'bmi': 1230.2753625796322, 'bp': -23.115972397773476, 's1': -2365.93678873304, 's2': -2575.9084327976316, 's3': 205.40949575640116, 's4': 872.8721525349347, 's5': 1225.332414583554, 's6': 528.2932637845697}


In [7]:
np.array(list(loci_importance.values()))

array([ -270.829268  ,    27.75608083,  1230.27536258,   -23.1159724 ,
       -2365.93678873, -2575.9084328 ,   205.40949576,   872.87215253,
        1225.33241458,   528.29326378])

In [4]:
import numpy as np

def toep (d, rho=0.6):
  return np.array([[ (rho)**abs(i-j) for i in range(d)]for j in range(d)])

def theoretical_curve(y_method, j, correlation,p, beta=[2, 1]):
    """
    Computes the theoretical value for a coordinate `j` based on the specified method.

    Parameters:
    -----------
    y_method : str
        The method used for computation. Can be either 'lin' (linear) or 'nonlin' (nonlinear).
    j : int
        The coordinate index for which the theoretical value is computed.
    correlation : float
        The correlation coefficient.
    p : int
        The dimension of the Toeplitz matrix used in the nonlinear case.
    beta : list, optional
        Coefficients used in the linear case, default is [2, 1].

    Returns:
    --------
    float
        The theoretical value for the given coordinate `j`.
    """
    if y_method == 'lin':
        return beta[j]**2*(1-correlation**2)
    elif y_method == 'fixed_poly':
        if j==0:
            mat=toep(p, correlation)
            sigma_0=mat[0]
            sigma_0=np.delete(sigma_0, 0)
            inv=np.delete(mat, 0, axis=0)
            inv=np.delete(inv, 0, axis=1)
            inv=np.linalg.inv(inv)
            return (correlation-np.dot(np.dot(sigma_0,inv), sigma_0.T))
        elif j==1:
            mat=toep(p, correlation)
            sigma_j=mat[j]
            sigma_j=np.delete(sigma_j, j)
            inv=np.delete(mat, j, axis=0)
            inv=np.delete(inv, j, axis=1)
            inv=np.linalg.inv(inv)
            return (4*(correlation-np.dot(np.dot(sigma_j,inv), sigma_j.T)))
        elif j==4:# var(X²) = 2sigma⁴+4sigma²mu²
            mat=toep(p, correlation)
            sigma_j=mat[j]
            sigma_j=np.delete(sigma_j, j)
            inv=np.delete(mat, j, axis=0)
            inv=np.delete(inv, j, axis=1)
            inv=np.linalg.inv(inv)
            cond_var= (correlation-np.dot(np.dot(sigma_j,inv), sigma_j.T))
            mu = np.zeros(p)
            Sigma = toep(p, correlation)  # covariance matrix of X
            rng = np.random.default_rng(0)
            X = rng.multivariate_normal(mu, Sigma, size=(10000))
            X_minus_j = np.delete(X, j, axis=1)
            mn=np.dot(X_minus_j, np.dot(sigma_j,inv))
            return np.mean(2*cond_var**2+4*cond_var*mn**2)
        elif j==7 or j==8:# sigma²*sigma_cond²
            mat=toep(p, correlation)
            sigma_j=mat[j]
            sigma_j=np.delete(sigma_j, j)
            inv=np.delete(mat, j, axis=0)
            inv=np.delete(inv, j, axis=1)
            inv=np.linalg.inv(inv)
            return correlation**2*(correlation-np.dot(np.dot(sigma_j,inv), sigma_j.T))
        else:
            return 0


In [5]:
#True values
true_values = {}
for j in range(10):
    true_values['V'+str(j)]=theoretical_curve('fixed_poly', j, 0.6,10)

In [6]:
true_values

{'V0': np.float64(0.23999999999999994),
 'V1': np.float64(0.2823529411764705),
 'V2': 0,
 'V3': 0,
 'V4': np.float64(0.15918653482012055),
 'V5': 0,
 'V6': 0,
 'V7': np.float64(0.025411764705882342),
 'V8': np.float64(0.02541176470588238),
 'V9': 0}

In [8]:
mat=toep(10, 0.6)
sigma_0=mat[0]
sigma_0=np.delete(sigma_0, 0)
inv=np.delete(mat, 0, axis=0)
inv=np.delete(inv, 0, axis=1)
inv=np.linalg.inv(inv)
(np.dot(np.dot(sigma_0,inv), sigma_0.T))

np.float64(0.36000000000000004)

In [13]:
mat[0]

array([1.        , 0.6       , 0.36      , 0.216     , 0.1296    ,
       0.07776   , 0.046656  , 0.0279936 , 0.01679616, 0.0100777 ])

In [12]:
sigma_0

array([0.6       , 0.36      , 0.216     , 0.1296    , 0.07776   ,
       0.046656  , 0.0279936 , 0.01679616, 0.0100777 ])